# 02 - Hedonic Indices (Reconstructed)

This notebook mirrors the original hedonic workflow using the curated datasets stored under `final_code/data/`. It documents the steps even though the production pipeline now lives inside the Python modules.

In [ ]:
from pathlib import Path
import pandas as pd
from final_code import config

main_df = pd.read_parquet(config.DATA_DIR / 'main_features.parquet')
mesh_features = pd.read_csv(config.DATA_DIR / 'mesh_quarter_features.csv')
print(f'Transactions loaded: {len(main_df):,}')
print(f'Mesh-quarter rows: {len(mesh_features):,}')

## Ward-level hedonic regression

In [ ]:
import numpy as np
import statsmodels.formula.api as smf

ward_sample = main_df.copy()
ward_sample['PeriodOrder'] = ward_sample['PeriodKey'].apply(lambda x: int(x.split('-Q')[0]) * 4 + int(x[-1]) - 1)
model = smf.ols('np.log(PricePerSqM + 1) ~ C(WardName) + C(PeriodKey)', data=ward_sample).fit()
model.summary().tables[0]

## Mesh-level snapshot

In [ ]:
mesh_panel = mesh_features.groupby(['Mesh250m','PeriodKey']).agg(
    median_price=('mesh_median_ppsqm','median'),
    count=('mesh_transaction_count','sum')
).reset_index()
mesh_panel.head()

### Notes
- This notebook is for documentation only; production hedonic indices are generated in `final_code/panels.py`.
- Keep it updated if you need to present intermediary hedonic plots in progress reports.